# 자연스러운 코드로

- 가변 길이 인수 (순전파)
- 가변 길이 인수 (개선)
- 가변 길이 인수 (역전파)


## 가변 길이 인수 (순전파)

```python
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs] # Variable 인스턴스로부터 데이터를 꺼낸다.
        ys = self.forward(xs) # forward 메서드에서 구체적인 계산을 수행한다.
        outputs = [Variable(as_array(y)) for y in ys] # 계산된 데이터를 Variable 인스턴스로 다시 감싼다.

        for output in outputs:
            output.set_creator(self) # 원산지 표시를 한다.

        self.inputs = inputs  # 입력 변수를 기억(보관)한다.
        self.outputs = outputs  # 출력 변수를 저장한다.
        return outputs

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()
```

인수와 반환값을 리스트로 변경했다.


### Add 클래스

```python
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)  # 반환값을 튜플로 묶는다.
```


In [ ]:
import numpy as np
from framework.variable import Variable
from framework.function import Add
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

## 가변 길이 인수 (개선)

### 함수를 사용하기 쉽게 개선

```python
class Function:
  def __call__(self, *inputs):  # *를 붙여 가변 길이 인수로 변경
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)  # 리스트 대신 언패킹하여 넘긴다.
    if not isinstance(ys, tuple):  # 튜플이 아닌 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]
```


In [ ]:
from framework.function import Add
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

In [3]:
from framework.function import add
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

inputs:  2
5
